In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot
from rt_analysis import rt_parser

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

# 100 Recs - D11

## 1 - Corridor Improvements

In [ ]:
def fm_from_bbutils(ct_dist, category, get_sorted = False,
                    fm_dict = None, analysis_date = dt.date(2022, 10, 4)):
    '''
    ct_dist: str, Caltrans district, formatted as in "11 - San Diego"
    category: str, reccomendation category, "corridor" or "hotspot"
    get_sorted: use better_bus_utils.get_sorted_transit_routes instead of 
    corridor/hotspot dedicated function. Default False, override if few/no
    suitable routes returned.
    fm_dict: dictionary of rt_analysis.rt_filter_map_plot.RtFilterMapper instances,
    if some are already loaded (for example when running for hotspot after corridor)
    '''
    
    if get_sorted:
        bbutil_gdf = better_bus_utils.get_sorted_transit_routes(recommendation_category=category)
    elif category == "corridor":
        bbutil_gdf = better_bus_utils.select_transit_routes_corridor_improvements()
    elif category == "hotspot":
        bbutil_gdf = better_bus_utils.select_transit_routes_hotspot_improvements()
    
    bbutil_gdf = bbutil_gdf >> filter(_.caltrans_district==ct_dist)
    if bbutil_gdf.empty:
        print(f"{ct_dist} {category} empty with default util, retrying with get_sorted")
        return fm_from_bbutils(ct_dist = ct_dist, category = category, get_sorted = True,
                              fm_dict = fm_dict, analysis_date = analysis_date)
    top_25_pct = bbutil_gdf.shape[0] // 5
    bbutil_gdf = bbutil_gdf >> head(top_25_pct) # display top 20% of suggested routes
    unique_itp_ids = bbutil_gdf.calitp_itp_id.unique()
    mappable = bbutil_gdf >> select(-_.service_date)
    if len(unique_itp_ids) == 1:
        display(mappable.explore("route_id", legend=True, tiles="CartoDB positron"))
    else:
        display(mappable.explore("calitp_itp_id", legend=True, tiles="CartoDB positron"))
    fm_dict = {}
    for itp_id in unique_itp_ids:
        print(f'loading filter/mapper for {itp_id}')
        fm = rt_filter_map_plot.from_gcs(itp_id=itp_id, analysis_date=analysis_date)
        rt_id_names = (fm.rt_trips
                 >> distinct(_.route_id, _.route_short_name)
                 >> inner_join(_, bbutil_gdf >> select(_.route_id), on = 'route_id')
                )
        display(rt_id_names)
        fm.set_filter(route_names = rt_id_names.route_short_name)
        fm_dict[itp_id] = fm
    print(fm_dict)
    return fm_dict

In [ ]:
fm_dict = fm_from_bbutils(ct_dist="07 - Los Angeles", category="corridor")

In [ ]:
import yaml

In [ ]:
with open('../portfolio/sites/rt.yml') as f:
    rt_site = yaml.safe_load(f)

In [ ]:
chapters = rt_site['parts'][0]['chapters']

In [ ]:
itp_ids = []

In [ ]:
for chapter in chapters:
    itp_ids += [section['itp_id'] for section in chapter['sections']]

In [ ]:
# itp_ids

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
pbar = tqdm()

In [ ]:
rt

In [ ]:
m = fm_dict[182].segment_speed_map()

In [ ]:
m

In [ ]:
# get small df of route_id and route_short_name from rt_trips
d11_routenames = (sdmts.rt_trips
                 >> count(_.route_id, _.route_short_name, _.route_long_name)
                 >> select(-_.n)
                )

d11_routenames >> head(5)

In [ ]:
# inner join w gdf of slow routes
corridor_d11_names = (corridor_d11
                     >> inner_join(_,d11_routenames)
                    )

corridor_d11_names >> head(3)

In [ ]:
# pass the route names as a filter parameter
routename_list_c = corridor_d11_names["route_short_name"].tolist()
routename_list_c

In [ ]:
sdmts.set_filter(start_time='06:00', end_time='09:00', route_names = routename_list_c)

In [ ]:
am = sdmts.segment_speed_map()

In [ ]:
am

In [ ]:
sdmts.set_filter(start_time='15:00', end_time='19:00', route_names = routename_list_c)

In [ ]:
pm = sdmts.segment_speed_map()

In [ ]:
pm

## Draft Corridor Recs, SDMTS, D11

* Entire Route 28 (nn mi, nn speed-based delay, nn schedule-based delay)
* Route 10, Goldfinch to 54 ()
    * paralleled by faster Rapid 215 E of Park
* Route 27, Mission to Moraga
* Route 12, Broadway to Euclid

## Route 28

In [ ]:
shape_id = '28_2_48'
stop_range = [2, 18]

In [ ]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
sdmts.reset_filter()

In [ ]:
sdmts.corridor_metrics()

In [ ]:
sdmts.quick_map_corridor()

In [ ]:
corr_28 = sdmts.corridor.copy()

In [ ]:
corridors = [corr_28]

In [ ]:
# sdmts.corridor.explore(m = pm)

## Route 10

In [ ]:
shape_id = '10_2_68'
stop_range = [6, 21]

In [ ]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
sdmts.reset_filter()

In [ ]:
sdmts.corridor_metrics()

In [ ]:
sdmts.quick_map_corridor()

In [ ]:
corr_10 = sdmts.corridor.copy()
corridors += [corr_10]

## Route 27

In [ ]:
shape_id = '27_3_87'
stop_range = [16, 31]

In [ ]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
sdmts.reset_filter()

In [ ]:
sdmts.corridor_metrics()

In [ ]:
sdmts.quick_map_corridor()

In [ ]:
corr_27 = sdmts.corridor.copy()
corridors += [corr_27]

## Route 12

In [ ]:
shape_id = '12_2_13'
stop_range = [3, 31]

In [ ]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
sdmts.set_filter(route_names=['12'])

In [ ]:
sdmts.corridor_metrics()

In [ ]:
sdmts.quick_map_corridor()

In [ ]:
corr_12 = sdmts.corridor.copy()
corridors += [corr_12]

# Corridor Map

In [ ]:
pd.concat(corridors).explore(m = pm)

## 2 - Hotspot Improvements

In [ ]:
hotspot_gdf_d11 = ((better_bus_utils.get_sorted_transit_routes(recommendation_category="hotspot"))
                   >>filter(_.caltrans_district=="11 - San Diego")
                   >>head(10) #get top 10
                  )

In [ ]:
# hotspot_gdf_d11

In [ ]:
hotspot_gdf_d11 = (hotspot_gdf_d11
               >> select(-_.service_date)
              )

hotspot_gdf_d11.explore("route_id", legend=True, tiles="CartoDB positron")

In [ ]:
# inner join w gdf of slow routes
hs_d11_names = (hotspot_gdf_d11
                     >> inner_join(_,d11_routenames)
                    )

hs_d11_names

In [ ]:
# pass the route names as a filter parameter
routename_list_hs = hs_d11_names["route_short_name"].tolist()
routename_list_hs

In [ ]:
sdmts.set_filter(start_time='06:00', end_time='09:00', route_names = routename_list_hs)

In [ ]:
am = sdmts.segment_speed_map()

In [ ]:
am

In [ ]:
sdmts.set_filter(start_time='15:00', end_time='19:00', route_names = routename_list_hs)

In [ ]:
pm = sdmts.segment_speed_map()

In [ ]:
pm

# Draft Hotspot Recs, SDMTS, D11

## Route 27 (Genesee)

In [ ]:
shape_id = '27_2_86'
stop_range = [19, 23]

In [ ]:
sdmts.reset_filter()

In [ ]:
sdmts.autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
sdmts.corridor_metrics()

In [ ]:
sdmts.quick_map_corridor()